In [ ]:
# load in packages
from itertools import combinations

#from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head(10)

In [ ]:
#x and y axes input data
y = train_data['Promotion']
x =  train_data.drop(['Promotion'], axis = 1)

In [ ]:
df = train_data.drop(['ID','purchase','Promotion'], axis = 1)

In [ ]:
# Split data into train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

xgbr = xgb.XGBRegressor()

params = {'learning_rate': [0.10,0.14,0.15,0.16, 0.2]}

xgbr_reg = GridSearchCV(xgbr, params, cv = 10, n_jobs =1)
xgbr_reg.fit(X_train, y_train)


#print("Best params:{}".format(xgbr_reg.best_params_))

#best_x = xgbr_reg.best_estimator_
y_train_pred_x = xgbr_reg.predict(X_train)
y_val_pred_x = xgbr_reg.predict(X_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# Import the necessary modules
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder
# #############################################################################

 # create regressor object 
regressor = RandomForestRegressor(n_estimators = 100, random_state = 42) 
regressor.fit(X_train, y_train)
# fit the regressor with x and y data 
   
# Compute and print metrics
scores = -cross_val_score(regressor, X_train, y_train, cv = 10, n_jobs = -1,scoring = 'neg_root_mean_squared_error')
# #############################################################################
print('RMSE scores mean: {:.3f}'.format(scores.mean()))
y_pred=regressor.predict(X_test)
 